## **WEEK 2**

##**PROJECT 2**

### **RFM ANALYSIS**

Recency, Frequency, and Monetary is a marketing technique used to quantitavely rank and group customers based on the recency, frequency and monetary total of their recent transactions to identify the best customers & perform targeted marketed campaigns.

Task
Perform RFM analysis and develop customer segments based on their RFM Scores
The analysis should provide insights into customer behaviour & identification of high-value customers, at-risk customers, & potential opportunities for personalized marketing campaigns



In [ ]:
!pip install h3pandas
!pip install h3
!pip install keplergl


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.2/115.2 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 9.6 MB/s eta 0:00:00
  Created wheel for h3pandas: filename=h3pandas-0.2.4-py3-none-any.whl size=17657 sha256=4399fa7bce1e37aa6f8bcb697969c187450c5a754377d0417040452a0cc2788c
  Stored in directory: /root/.cache/pip/wheels/b7/c1/a1/cfc5bbcd909a037e628ab0bb3717f72f9a2e599f0cfb322520
Successfully built h3pandas
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 42.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached jedi-0.19.1-py2.py3-none-any.whl (1.6 MB)
  Created wheel for keplergl: filename=keplergl-0.3.2-py2.py3-none-any.whl size=17922316 sha256=550459decdeb0edd1f1ea924c5d6edc9b8c14c58a80b025bf5ccc85aea37cc30
  Stored in directory: /root/.cache/pip/wheels/e5/e9/db/b8b8f2fcb5cb1cd7268d889

In [5]:
# Load important libraries
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.io as pio
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objects as go
pio.templates.default = "plotly_white"
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import Image
import branca.colormap
import folium
# since i would like to get a geopraphical representation of customers rfm score , we will import
# the following libraries
import h3
import h3pandas
import keplergl
import ipywidgets
from keplergl import KeplerGl


In [6]:
# Read data
df = pd.read_csv('/content/drive/MyDrive/Bootcamp data files/rfm_data.csv')
df.head()

,CustomerID,PurchaseDate,TransactionAmount,ProductInformation,OrderID,Location
0,8814,2023-04-11,943.31,Product C,890075,Tokyo
1,2188,2023-04-11,463.70,Product A,176819,London
2,4608,2023-04-11,80.28,Product A,340062,New York
3,2559,2023-04-11,221.29,Product A,239145,London
4,9482,2023-04-11,739.56,Product A,194545,Paris


import numpy as np
from google.colab import autoviz

def value_plot(df, y, figscale=1):
  from matplotlib import pyplot as plt
  df[y].plot(kind='line', figsize=(8 * figscale, 4 * figscale), title=y)
  plt.gca().spines[['top', 'right']].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = value_plot(_df_0, *['CustomerID'], **{})
chart

import numpy as np
from google.colab import autoviz

def value_plot(df, y, figscale=1):
  from matplotlib import pyplot as plt
  df[y].plot(kind='line', figsize=(8 * figscale, 4 * figscale), title=y)
  plt.gca().spines[['top', 'right']].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = value_plot(_df_1, *['TransactionAmount'], **{})
chart

import numpy as np
from google.colab import autoviz

def value_plot(df, y, figscale=1):
  from matplotlib import pyplot as plt
  df[y].plot(kind='line', figsize=(8 * figscale, 4 * figscale), title=y)
  plt.gca().spines[['top', 'right']].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = value_plot(_df_2, *['OrderID'], **{})
chart

import numpy as np
from google.colab import autoviz

def histogram(df, colname, num_bins=20, figscale=1):
  from matplotlib import pyplot as plt
  df[colname].plot(kind='hist', bins=num_bins, title=colname, figsize=(8*figscale, 4*figscale))
  plt.gca().spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = histogram(_df_3, *['CustomerID'], **{})
chart

...

import numpy as np
from google.colab import autoviz

def violin_plot(df, value_colname, facet_colname, figscale=1, mpl_palette_name='Dark2', **kwargs):
  from matplotlib import pyplot as plt
  import seaborn as sns
  figsize = (12 * figscale, 1.2 * figscale * len(df[facet_colname].unique()))
  plt.figure(figsize=figsize)
  sns.violinplot(df, x=value_colname, y=facet_colname, palette=mpl_palette_name, **kwargs)
  sns.despine(top=True, right=True, bottom=True, left=True)
  return autoviz.MplChart.from_current_mpl_state()

chart = violin_plot(_df_13, *['TransactionAmount', 'Location'], **{'inner': 'stick'})
chart

import numpy as np
from google.colab import autoviz

def time_series_multiline(df, timelike_colname, value_colname, series_colname, figscale=1, mpl_palette_name='Dark2'):
  from matplotlib import pyplot as plt
  import seaborn as sns
  figsize = (10 * figscale, 5.2 * figscale)
  palette = list(sns.palettes.mpl_palette(mpl_palette_name))
  def _plot_series(series, series_name, series_index=0):
    if value_colname == 'count()':
      counted = (series[timelike_colname]
                 .value_counts()
                 .reset_index(name='counts')
                 .rename({'index': timelike_colname}, axis=1)
                 .sort_values(timelike_colname, ascending=True))
      xs = counted[timelike_colname]
      ys = counted['counts']
    else:
      xs = series[timelike_colname]
      ys = series[value_colname]
    plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

  fig, ax = plt.subplots(figsize=figsize, layout='constrained')
  df = df.sort_values(timelike_colname, ascending=True)
  if series_colname:
    for i, (series_name, series) in enumerate(df.groupby(series_colname)):
      _plot_series(series, series_name, i)
    fig.legend(title=series_colname, bbox_to_anchor=(1, 1), loc='upper left')
  else:
    _plot_series(df, '')
  sns.despine(fig=fig, ax=ax)
  plt.xlabel(timelike_colname)
  plt.ylabel(value_colname)
  return autoviz.MplChart.from_current_mpl_state()

chart = time_series_multiline(_df_14, *['PurchaseDate', 'CustomerID', 'ProductInformation'], **{})
chart

import numpy as np
from google.colab import autoviz

def time_series_multiline(df, timelike_colname, value_colname, series_colname, figscale=1, mpl_palette_name='Dark2'):
  from matplotlib import pyplot as plt
  import seaborn as sns
  figsize = (10 * figscale, 5.2 * figscale)
  palette = list(sns.palettes.mpl_palette(mpl_palette_name))
  def _plot_series(series, series_name, series_index=0):
    if value_colname == 'count()':
      counted = (series[timelike_colname]
                 .value_counts()
                 .reset_index(name='counts')
                 .rename({'index': timelike_colname}, axis=1)
                 .sort_values(timelike_colname, ascending=True))
      xs = counted[timelike_colname]
      ys = counted['counts']
    else:
      xs = series[timelike_colname]
      ys = series[value_colname]
    plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

  fig, ax = plt.subplots(figsize=figsize, layout='constrained')
  df = df.sort_values(timelike_colname, ascending=True)
  if series_colname:
    for i, (series_name, series) in enumerate(df.groupby(series_colname)):
      _plot_series(series, series_name, i)
    fig.legend(title=series_colname, bbox_to_anchor=(1, 1), loc='upper left')
  else:
    _plot_series(df, '')
  sns.despine(fig=fig, ax=ax)
  plt.xlabel(timelike_colname)
  plt.ylabel(value_colname)
  return autoviz.MplChart.from_current_mpl_state()

chart = time_series_multiline(_df_15, *['PurchaseDate', 'CustomerID', 'Location'], **{})
chart

import numpy as np
from google.colab import autoviz

def time_series_multiline(df, timelike_colname, value_colname, series_colname, figscale=1, mpl_palette_name='Dark2'):
  from matplotlib import pyplot as plt
  import seaborn as sns
  figsize = (10 * figscale, 5.2 * figscale)
  palette = list(sns.palettes.mpl_palette(mpl_palette_name))
  def _plot_series(series, series_name, series_index=0):
    if value_colname == 'count()':
      counted = (series[timelike_colname]
                 .value_counts()
                 .reset_index(name='counts')
                 .rename({'index': timelike_colname}, axis=1)
                 .sort_values(timelike_colname, ascending=True))
      xs = counted[timelike_colname]
      ys = counted['counts']
    else:
      xs = series[timelike_colname]
      ys = series[value_colname]
    plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

  fig, ax = plt.subplots(figsize=figsize, layout='constrained')
  df = df.sort_values(timelike_colname, ascending=True)
  if series_colname:
    for i, (series_name, series) in enumerate(df.groupby(series_colname)):
      _plot_series(series, series_name, i)
    fig.legend(title=series_colname, bbox_to_anchor=(1, 1), loc='upper left')
  else:
    _plot_series(df, '')
  sns.despine(fig=fig, ax=ax)
  plt.xlabel(timelike_colname)
  plt.ylabel(value_colname)
  return autoviz.MplChart.from_current_mpl_state()

chart = time_series_multiline(_df_16, *['PurchaseDate', 'TransactionAmount', 'ProductInformation'], **{})
chart

import numpy as np
from google.colab import autoviz

def time_series_multiline(df, timelike_colname, value_colname, series_colname, figscale=1, mpl_palette_name='Dark2'):
  from matplotlib import pyplot as plt
  import seaborn as sns
  figsize = (10 * figscale, 5.2 * figscale)
  palette = list(sns.palettes.mpl_palette(mpl_palette_name))
  def _plot_series(series, series_name, series_index=0):
    if value_colname == 'count()':
      counted = (series[timelike_colname]
                 .value_counts()
                 .reset_index(name='counts')
                 .rename({'index': timelike_colname}, axis=1)
                 .sort_values(timelike_colname, ascending=True))
      xs = counted[timelike_colname]
      ys = counted['counts']
    else:
      xs = series[timelike_colname]
      ys = series[value_colname]
    plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

  fig, ax = plt.subplots(figsize=figsize, layout='constrained')
  df = df.sort_values(timelike_colname, ascending=True)
  if series_colname:
    for i, (series_name, series) in enumerate(df.groupby(series_colname)):
      _plot_series(series, series_name, i)
    fig.legend(title=series_colname, bbox_to_anchor=(1, 1), loc='upper left')
  else:
    _plot_series(df, '')
  sns.despine(fig=fig, ax=ax)
  plt.xlabel(timelike_colname)
  plt.ylabel(value_colname)
  return autoviz.MplChart.from_current_mpl_state()

chart = time_series_multiline(_df_17, *['PurchaseDate', 'TransactionAmount', 'Location'], **{})
chart

In [8]:
df.info


<bound method DataFrame.info of      CustomerID PurchaseDate  TransactionAmount ProductInformation  OrderID  \
0          8814   2023-04-11             943.31          Product C   890075   
1          2188   2023-04-11             463.70          Product A   176819   
2          4608   2023-04-11              80.28          Product A   340062   
3          2559   2023-04-11             221.29          Product A   239145   
4          9482   2023-04-11             739.56          Product A   194545   
..          ...          ...                ...                ...      ...   
995        2970   2023-06-10             759.62          Product B   275284   
996        6669   2023-06-10             941.50          Product C   987025   
997        8836   2023-06-10             545.36          Product C   512842   
998        1440   2023-06-10             729.94          Product B   559753   
999        4759   2023-06-10             804.28          Product D   467544   

     Location  
0  

In [11]:
df.info(['ProductInformation'])


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   CustomerID          1000 non-null   int64  
 1   PurchaseDate        1000 non-null   object 
 2   TransactionAmount   1000 non-null   float64
 3   ProductInformation  1000 non-null   object 
 4   OrderID             1000 non-null   int64  
 5   Location            1000 non-null   object 
dtypes: float64(1), int64(2), object(3)
memory usage: 47.0+ KB


In [12]:
df.describe()

,CustomerID,TransactionAmount,OrderID
count,1000.000000,1000.00000,1000.000000
mean,5554.789000,513.67781,554071.398000
std,2605.014863,286.09870,264695.448814
min,1011.000000,12.13000,100096.000000
25%,3273.000000,257.12000,313152.000000
50%,5538.000000,523.56500,564671.500000
75%,7821.750000,759.86000,783052.250000
max,9991.000000,999.44000,999695.000000


In [13]:
#Lets tansform customerid to object same case to orderid
df['CustomerID'] = df['CustomerID'].astype(str)
df['OrderID'] = df['OrderID'].astype(str)

In [14]:
print(df.dtypes)

CustomerID             object
PurchaseDate           object
TransactionAmount     float64
ProductInformation     object
OrderID                object
Location               object
dtype: object


In [16]:
product_types = df['ProductInformation'].unique()
print(product_types)

['Product C' 'Product A' 'Product B' 'Product D']


## Data Preprocessing


* Hndle missing values, outliers and any inconsistencies in the data
* Calculate the recency, frequency, and monetary value for each customer based on their transaction history.



In [15]:
# Check for missing values, outliers and any inconsistencies
df.isnull().sum()

CustomerID            0
PurchaseDate          0
TransactionAmount     0
ProductInformation    0
OrderID               0
Location              0
dtype: int64

In [17]:
df.describe()

,TransactionAmount
count,1000.00000
mean,513.67781
std,286.09870
min,12.13000
25%,257.12000
50%,523.56500
75%,759.86000
max,999.44000


import numpy as np
from google.colab import autoviz

def value_plot(df, y, figscale=1):
  from matplotlib import pyplot as plt
  df[y].plot(kind='line', figsize=(8 * figscale, 4 * figscale), title=y)
  plt.gca().spines[['top', 'right']].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = value_plot(_df_18, *['TransactionAmount'], **{})
chart

import numpy as np
from google.colab import autoviz

def histogram(df, colname, num_bins=20, figscale=1):
  from matplotlib import pyplot as plt
  df[colname].plot(kind='hist', bins=num_bins, title=colname, figsize=(8*figscale, 4*figscale))
  plt.gca().spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = histogram(_df_19, *['TransactionAmount'], **{})
chart

In [ ]:
df

In [20]:
df['PurchaseDate'].dtype

dtype('<M8[ns]')

**Calculating RFM Values**

In [21]:
from datetime import datetime

# Convert PurchaseDate to datetime
df['PurchaseDate'] = pd.to_datetime(df['PurchaseDate'])

# Calculate Recency
current_date = datetime.now().date()
df['Recency'] = (current_date - df['PurchaseDate'].dt.date).dt.days

# Calculate Frequency
frequency_df = df.groupby('CustomerID')['OrderID'].count().reset_index()
frequency_df.rename(columns={'OrderID': 'Frequency'}, inplace=True)
df = df.merge(frequency_df, on='CustomerID', how='left')

# Calculate Monetary value
monetary_df = df.groupby('CustomerID')['TransactionAmount'].sum().reset_index()
monetary_df.rename(columns={'TransactionAmount': 'MonetaryValue'}, inplace=True)
df = df.merge(monetary_df, on='CustomerID', how='left')


In [23]:
print(df.head())

  CustomerID PurchaseDate  TransactionAmount ProductInformation OrderID  \
0       8814   2023-04-11             943.31          Product C  890075   
1       2188   2023-04-11             463.70          Product A  176819   
2       4608   2023-04-11              80.28          Product A  340062   
3       2559   2023-04-11             221.29          Product A  239145   
4       9482   2023-04-11             739.56          Product A  194545   

   Location  Recency  Frequency  MonetaryValue  
0     Tokyo      177          1         943.31  
1    London      177          1         463.70  
2  New York      177          1          80.28  
3    London      177          1         221.29  
4     Paris      177          1         739.56  


## RFM Score Calculation


We will calculate RFM scores for each customer

In [24]:
# Define scoring criteria for each RFM value
recency_scores = [5, 4, 3, 2, 1] # Higher score for lower recency (more recent)
frequency_scores = [1, 2, 3, 4, 5] # Higher score for higher frequency
monetary_scores = [1, 2, 3, 4, 5]  # Higher score for high monetary value

# Calculate RFM scores
df['RecencyScore'] = pd.cut(df['Recency'], bins=5, labels=recency_scores)
df['FrequencyScore'] = pd.cut(df['Frequency'], bins=5, labels=frequency_scores)
df['MonetaryScore'] = pd.cut(df['MonetaryValue'], bins=5, labels=monetary_scores)

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 0 to 999
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   CustomerID          1000 non-null   object        
 1   PurchaseDate        1000 non-null   datetime64[ns]
 2   TransactionAmount   1000 non-null   float64       
 3   ProductInformation  1000 non-null   object        
 4   OrderID             1000 non-null   object        
 5   Location            1000 non-null   object        
 6   Recency             1000 non-null   int64         
 7   Frequency           1000 non-null   int64         
 8   MonetaryValue       1000 non-null   float64       
 9   RecencyScore        1000 non-null   category      
 10  FrequencyScore      1000 non-null   category      
 11  MonetaryScore       1000 non-null   category      
dtypes: category(3), datetime64[ns](1), float64(2), int64(2), object(4)
memory usage: 81.7+ KB


In [27]:
# Convert RFM scores to numeric type
df['RecencyScore'] = df['RecencyScore'].astype(int)
df['FrequencyScore'] = df['FrequencyScore'].astype(int)
df['MonetaryScore'] = df['MonetaryScore'].astype(int)

## RFM Segmentation
### Quantile-Based Segmentation
Divide customers into quartiles or quintiles based on their RFM scores. This creates predefine segments like"High-Value", "Mid-Value", and "Low-Value" customers

In [28]:
# Combine individual score
df['RFM_Score'] = df['RecencyScore'] + df['FrequencyScore'] + df['MonetaryScore']

segment_labels = ['Low-Value', 'Mid-Value', 'High-Value']
df['Segment Value'] = pd.qcut(df['RFM_Score'], q=3, labels=segment_labels)


In [29]:
print(df.head())

  CustomerID PurchaseDate  TransactionAmount ProductInformation OrderID  \
0       8814   2023-04-11             943.31          Product C  890075   
1       2188   2023-04-11             463.70          Product A  176819   
2       4608   2023-04-11              80.28          Product A  340062   
3       2559   2023-04-11             221.29          Product A  239145   
4       9482   2023-04-11             739.56          Product A  194545   

   Location  Recency  Frequency  MonetaryValue  RecencyScore  FrequencyScore  \
0     Tokyo      177          1         943.31             1               1   
1    London      177          1         463.70             1               1   
2  New York      177          1          80.28             1               1   
3    London      177          1         221.29             1               1   
4     Paris      177          1         739.56             1               1   

   MonetaryScore  RFM_Score Segment Value  
0              2        

In [30]:
# RFM Segmentation Distribution
segments_counts = df['Segment Value'].value_counts().reset_index()
segments_counts.columns = ['Segment Value', 'Count']

pastel_colors = px.colors.qualitative.Pastel

# Bar chart
fig_segment_dist = px.bar(segments_counts, x='Segment Value', y='Count',
                          color='Segment Value', color_discrete_sequence=pastel_colors,
                          title='RFM Value Segment Distribution')
fig_segment_dist.update_layout(xaxis_title = 'RFM Value Segment',
                               yaxis_title='Count',
                               showlegend=False)
fig_segment_dist.show()

RFM Customer segments

Let's create and analyze RFM Customer Segments that are broader classifications based on the RFM scores.i.e champions, Potential Loyalists and Can't Lose


In [31]:
# Create a new column for RFM Customer Segments
df['RFM Customer Segments'] = ''

# Assign RFM segments based on the RFM score
df.loc[df['RFM_Score'] >= 9, 'RFM Customer Segments'] = 'Champions'
df.loc[(df['RFM_Score'] >= 6) & (df['RFM_Score'] < 9), 'RFM Customer Segments'] = 'Potential Loyalists'
df.loc[(df['RFM_Score'] >= 5) & (df['RFM_Score'] < 6), 'RFM Customer Segments'] = 'At Risk Customers'
df.loc[(df['RFM_Score'] >= 4) & (df['RFM_Score'] < 5), 'RFM Customer Segments'] = "Can't Lose"
df.loc[(df['RFM_Score'] >= 3) & (df['RFM_Score'] < 4), 'RFM Customer Segments'] = "Lost"

# Print the updated data with RFM segments
print(df[['CustomerID', 'RFM Customer Segments']])

    CustomerID RFM Customer Segments
0         8814            Can't Lose
1         2188                  Lost
2         4608                  Lost
3         2559                  Lost
4         9482            Can't Lose
..         ...                   ...
995       2970   Potential Loyalists
996       6669   Potential Loyalists
997       8836   Potential Loyalists
998       1440   Potential Loyalists
999       4759   Potential Loyalists

[1000 rows x 2 columns]


### RFM Analysis
Let's analyze the distribution of customers across different RFM customer segments within each value

In [32]:
df.head(10)

,CustomerID,PurchaseDate,TransactionAmount,ProductInformation,OrderID,Location,Recency,Frequency,MonetaryValue,RecencyScore,FrequencyScore,MonetaryScore,RFM_Score,Segment Value,RFM Customer Segments
0,8814,2023-04-11,943.31,Product C,890075,Tokyo,177,1,943.31,1,1,2,4,Low-Value,Can't Lose
1,2188,2023-04-11,463.70,Product A,176819,London,177,1,463.70,1,1,1,3,Low-Value,Lost
2,4608,2023-04-11,80.28,Product A,340062,New York,177,1,80.28,1,1,1,3,Low-Value,Lost
3,2559,2023-04-11,221.29,Product A,239145,London,177,1,221.29,1,1,1,3,Low-Value,Lost
4,9482,2023-04-11,739.56,Product A,194545,Paris,177,1,739.56,1,1,2,4,Low-Value,Can't Lose
5,8483,2023-04-11,375.23,Product C,691194,Paris,177,1,375.23,1,1,1,3,Low-Value,Lost
6,8317,2023-04-11,272.56,Product B,826847,New York,177,2,974.88,1,3,3,7,Mid-Value,Potential Loyalists
7,6911,2023-04-11,433.33,Product C,963918,Tokyo,177,1,433.33,1,1,1,3,Low-Value,Lost
8,8993,2023-04-12,16.55,Product D,112426,New York,176,1,16.55,1,1,1,3,Low-Value,Lost
9,3519,2023-04-12,464.63,Product C,139726,New York,176,1,464.63,1,1,1,3,Low-Value,Lost


import numpy as np
from google.colab import autoviz

def value_plot(df, y, figscale=1):
  from matplotlib import pyplot as plt
  df[y].plot(kind='line', figsize=(8 * figscale, 4 * figscale), title=y)
  plt.gca().spines[['top', 'right']].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = value_plot(_df_20, *['TransactionAmount'], **{})
chart

import numpy as np
from google.colab import autoviz

def value_plot(df, y, figscale=1):
  from matplotlib import pyplot as plt
  df[y].plot(kind='line', figsize=(8 * figscale, 4 * figscale), title=y)
  plt.gca().spines[['top', 'right']].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = value_plot(_df_21, *['Recency'], **{})
chart

import numpy as np
from google.colab import autoviz

def value_plot(df, y, figscale=1):
  from matplotlib import pyplot as plt
  df[y].plot(kind='line', figsize=(8 * figscale, 4 * figscale), title=y)
  plt.gca().spines[['top', 'right']].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = value_plot(_df_22, *['Frequency'], **{})
chart

import numpy as np
from google.colab import autoviz

def value_plot(df, y, figscale=1):
  from matplotlib import pyplot as plt
  df[y].plot(kind='line', figsize=(8 * figscale, 4 * figscale), title=y)
  plt.gca().spines[['top', 'right']].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = value_plot(_df_23, *['MonetaryValue'], **{})
chart

...

import numpy as np
from google.colab import autoviz

def violin_plot(df, value_colname, facet_colname, figscale=1, mpl_palette_name='Dark2', **kwargs):
  from matplotlib import pyplot as plt
  import seaborn as sns
  figsize = (12 * figscale, 1.2 * figscale * len(df[facet_colname].unique()))
  plt.figure(figsize=figsize)
  sns.violinplot(df, x=value_colname, y=facet_colname, palette=mpl_palette_name, **kwargs)
  sns.despine(top=True, right=True, bottom=True, left=True)
  return autoviz.MplChart.from_current_mpl_state()

chart = violin_plot(_df_37, *['Recency', 'ProductInformation'], **{'inner': 'stick'})
chart

import numpy as np
from google.colab import autoviz

def time_series_multiline(df, timelike_colname, value_colname, series_colname, figscale=1, mpl_palette_name='Dark2'):
  from matplotlib import pyplot as plt
  import seaborn as sns
  figsize = (10 * figscale, 5.2 * figscale)
  palette = list(sns.palettes.mpl_palette(mpl_palette_name))
  def _plot_series(series, series_name, series_index=0):
    if value_colname == 'count()':
      counted = (series[timelike_colname]
                 .value_counts()
                 .reset_index(name='counts')
                 .rename({'index': timelike_colname}, axis=1)
                 .sort_values(timelike_colname, ascending=True))
      xs = counted[timelike_colname]
      ys = counted['counts']
    else:
      xs = series[timelike_colname]
      ys = series[value_colname]
    plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

  fig, ax = plt.subplots(figsize=figsize, layout='constrained')
  df = df.sort_values(timelike_colname, ascending=True)
  if series_colname:
    for i, (series_name, series) in enumerate(df.groupby(series_colname)):
      _plot_series(series, series_name, i)
    fig.legend(title=series_colname, bbox_to_anchor=(1, 1), loc='upper left')
  else:
    _plot_series(df, '')
  sns.despine(fig=fig, ax=ax)
  plt.xlabel(timelike_colname)
  plt.ylabel(value_colname)
  return autoviz.MplChart.from_current_mpl_state()

chart = time_series_multiline(_df_38, *['PurchaseDate', 'TransactionAmount', 'ProductInformation'], **{})
chart

import numpy as np
from google.colab import autoviz

def time_series_multiline(df, timelike_colname, value_colname, series_colname, figscale=1, mpl_palette_name='Dark2'):
  from matplotlib import pyplot as plt
  import seaborn as sns
  figsize = (10 * figscale, 5.2 * figscale)
  palette = list(sns.palettes.mpl_palette(mpl_palette_name))
  def _plot_series(series, series_name, series_index=0):
    if value_colname == 'count()':
      counted = (series[timelike_colname]
                 .value_counts()
                 .reset_index(name='counts')
                 .rename({'index': timelike_colname}, axis=1)
                 .sort_values(timelike_colname, ascending=True))
      xs = counted[timelike_colname]
      ys = counted['counts']
    else:
      xs = series[timelike_colname]
      ys = series[value_colname]
    plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

  fig, ax = plt.subplots(figsize=figsize, layout='constrained')
  df = df.sort_values(timelike_colname, ascending=True)
  if series_colname:
    for i, (series_name, series) in enumerate(df.groupby(series_colname)):
      _plot_series(series, series_name, i)
    fig.legend(title=series_colname, bbox_to_anchor=(1, 1), loc='upper left')
  else:
    _plot_series(df, '')
  sns.despine(fig=fig, ax=ax)
  plt.xlabel(timelike_colname)
  plt.ylabel(value_colname)
  return autoviz.MplChart.from_current_mpl_state()

chart = time_series_multiline(_df_39, *['PurchaseDate', 'TransactionAmount', 'Location'], **{})
chart

import numpy as np
from google.colab import autoviz

def time_series_multiline(df, timelike_colname, value_colname, series_colname, figscale=1, mpl_palette_name='Dark2'):
  from matplotlib import pyplot as plt
  import seaborn as sns
  figsize = (10 * figscale, 5.2 * figscale)
  palette = list(sns.palettes.mpl_palette(mpl_palette_name))
  def _plot_series(series, series_name, series_index=0):
    if value_colname == 'count()':
      counted = (series[timelike_colname]
                 .value_counts()
                 .reset_index(name='counts')
                 .rename({'index': timelike_colname}, axis=1)
                 .sort_values(timelike_colname, ascending=True))
      xs = counted[timelike_colname]
      ys = counted['counts']
    else:
      xs = series[timelike_colname]
      ys = series[value_colname]
    plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

  fig, ax = plt.subplots(figsize=figsize, layout='constrained')
  df = df.sort_values(timelike_colname, ascending=True)
  if series_colname:
    for i, (series_name, series) in enumerate(df.groupby(series_colname)):
      _plot_series(series, series_name, i)
    fig.legend(title=series_colname, bbox_to_anchor=(1, 1), loc='upper left')
  else:
    _plot_series(df, '')
  sns.despine(fig=fig, ax=ax)
  plt.xlabel(timelike_colname)
  plt.ylabel(value_colname)
  return autoviz.MplChart.from_current_mpl_state()

chart = time_series_multiline(_df_40, *['PurchaseDate', 'TransactionAmount', 'RFM Customer Segments'], **{})
chart

import numpy as np
from google.colab import autoviz

def time_series_multiline(df, timelike_colname, value_colname, series_colname, figscale=1, mpl_palette_name='Dark2'):
  from matplotlib import pyplot as plt
  import seaborn as sns
  figsize = (10 * figscale, 5.2 * figscale)
  palette = list(sns.palettes.mpl_palette(mpl_palette_name))
  def _plot_series(series, series_name, series_index=0):
    if value_colname == 'count()':
      counted = (series[timelike_colname]
                 .value_counts()
                 .reset_index(name='counts')
                 .rename({'index': timelike_colname}, axis=1)
                 .sort_values(timelike_colname, ascending=True))
      xs = counted[timelike_colname]
      ys = counted['counts']
    else:
      xs = series[timelike_colname]
      ys = series[value_colname]
    plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

  fig, ax = plt.subplots(figsize=figsize, layout='constrained')
  df = df.sort_values(timelike_colname, ascending=True)
  if series_colname:
    for i, (series_name, series) in enumerate(df.groupby(series_colname)):
      _plot_series(series, series_name, i)
    fig.legend(title=series_colname, bbox_to_anchor=(1, 1), loc='upper left')
  else:
    _plot_series(df, '')
  sns.despine(fig=fig, ax=ax)
  plt.xlabel(timelike_colname)
  plt.ylabel(value_colname)
  return autoviz.MplChart.from_current_mpl_state()

chart = time_series_multiline(_df_41, *['PurchaseDate', 'Recency', 'ProductInformation'], **{})
chart

In [33]:
df.info

<bound method DataFrame.info of     CustomerID PurchaseDate  TransactionAmount ProductInformation OrderID  \
0         8814   2023-04-11             943.31          Product C  890075   
1         2188   2023-04-11             463.70          Product A  176819   
2         4608   2023-04-11              80.28          Product A  340062   
3         2559   2023-04-11             221.29          Product A  239145   
4         9482   2023-04-11             739.56          Product A  194545   
..         ...          ...                ...                ...     ...   
995       2970   2023-06-10             759.62          Product B  275284   
996       6669   2023-06-10             941.50          Product C  987025   
997       8836   2023-06-10             545.36          Product C  512842   
998       1440   2023-06-10             729.94          Product B  559753   
999       4759   2023-06-10             804.28          Product D  467544   

     Location  Recency  Frequency  Monetary

Distibution of customer segments interms of different locations

In [35]:
df.groupby(['Location',])['RFM Customer Segments'].count()

Location
London      246
New York    247
Paris       229
Tokyo       278
Name: RFM Customer Segments, dtype: int64

In [37]:
# Stacked bar chart
segment_dist = df.groupby(['Segment Value', 'RFM Customer Segments'])['CustomerID'].count().reset_index()
fig = px.bar(segment_dist, x='Segment Value', y='CustomerID', color='RFM Customer Segments',
             title='Customer Distribution by RFM Segments within Value Segments',
             labels={'Segment Value': 'Segment Value', 'CustomerID': 'Customer Count', 'RFM Customer Segments': 'RFM Segments'})
fig.show()

In [41]:
segment_product_counts = df.groupby(['Segment Value', 'RFM Customer Segments']).size().reset_index(name='Count')
segment_product_counts = segment_product_counts.sort_values('Count', ascending=False)

# Define custom colors
custom_colors = {
    'High-Value':'#ADD8E6',
    'Mid-Value':'#800080',
    'Low-Value':'#FFC0CB',
}

fig_treemap = px.treemap(segment_product_counts,
                         path=['Segment Value', 'RFM Customer Segments'],
                         values='Count',
                         color='Segment Value',
                         color_discrete_map=custom_colors,
                         title='RFM Customer Segments by Value')
fig_treemap.show()

Let's analyze the distribution of RFM Values within the Champions segment


In [42]:
#Filter the dat to include only the customers in the Champions segment
champions_segment = df[df['RFM Customer Segments'] == 'Champions']

fig = go.Figure()
fig.add_trace(go.Box(y=champions_segment['RecencyScore'], name='Recency'))
fig.add_trace(go.Box(y=champions_segment['FrequencyScore'], name='Frequency'))
fig.add_trace(go.Box(y=champions_segment['MonetaryScore'], name='Monetary'))

fig.update_layout(title='Distribution of RFM Values within Champions Segment',
                  yaxis_title='RFM Value',
                  showlegend=True)
fig.show()

Let's analyze the correlation of the recency, frequency, and monetary scoes within the champions segment.

In [43]:
correlation_matrix = champions_segment[['RecencyScore', 'FrequencyScore', 'MonetaryScore']].corr()

print(correlation_matrix)

# Heatmap
fig_heatmap = px.imshow(correlation_matrix, x=['RecencyScore', 'FrequencyScore', 'MonetaryScore'],
                        y=['RecencyScore', 'Frequency', 'MonetaryScore'],
                        color_continuous_scale='Viridis', title='Correlation Heatmap (Champions)')
# Show Heatmap
fig_heatmap.show()

                RecencyScore  FrequencyScore  MonetaryScore
RecencyScore        1.000000       -0.571727      -0.474715
FrequencyScore     -0.571727        1.000000       0.390657
MonetaryScore      -0.474715        0.390657       1.000000


Let's look at the number of customers in all the segements

In [44]:
# Group by RFM Segments and count the number of customers in each segment
customer_seg_counts = df['RFM Customer Segments'].value_counts().reset_index()
customer_seg_counts.columns = ['RFM Customer Segments', 'Count']

# Define custom colors for each segment
custom_colors = {
    'Champions': 'blue',
    'Potential Loyalists':'green',
    'At Risk Customers':'yellow',
    'Can\'t Lose':'orange',
    'Lost':'red'
}

# Create a bar chart
fig_bar = px.bar(customer_seg_counts, x='RFM Customer Segments', y='Count',
                 color='RFM Customer Segments', color_discrete_map=custom_colors,
                 title='Number of Customers in Each RFM Segment',
                 labels={'RFM Customer Segments':'RFM Segments', 'Count':'Customer Count'})
fig_bar.show()

Let's have a look at the recency, frequency, and monetary scores of all segments

In [46]:
# Lets create the bar chart first without averaging the scores
# Melt the DataFrame to have the score in a single column for grouping
df_melted = pd.melt(df, id_vars=['RFM Customer Segments'], var_name='RFM Category', value_name='Score')

# Create a grouped bar chart
fig_grouped = px.bar(df_melted, x='RFM Customer Segments', y='Score', color='RFM Category',
                     title='RFM Scores by Segment',
                     labels={'RFM Customer Segemnts':'RFM Segments', 'Score':'RFM Score'},
                     category_orders={'RFM Category':['RecencyScore', 'FrequencyScore', 'MonetaryScore']})
fig_grouped.show()

Our bar graph is redudant , lets average the scores and create

In [47]:
# Calculate the average scores for each segment
df['Average_Recency'] = df['RecencyScore'].mean()
df['Average_Frequency'] = df['FrequencyScore'].mean()
df['Average_Monetary'] = df['MonetaryScore'].mean()

# Melt the DataFrame to have average score in a single column for grouping
df_melted = pd.melt(df, id_vars=['RFM Customer Segments'], var_name='RFM Category', value_name='Average_Score')

# Define custom colors for each RFM category
color_mapping = {
    'Average_Recency': 'blue',
    'Average_Frequency': 'green',
    'Average_Monetary': 'red'
}

# create a grouped bar chart
fig = px.bar(df_melted, x='RFM Customer Segments', y='Average_Score', color='RFM Category',
             title='Average RFM Scores by Segment',
             labels={'RFM Customer Segments': 'RFM Segment', 'Average_Score':'Average RFM Score'},
             color_discrete_sequence=color_mapping,
             category_orders={'RFM Category': ['Average_Recency', 'Average_Frequency', 'Average_Monetary']})

fig.show()

KeyError: ignored